In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
# prepare data
bc = datasets.load_breast_cancer()
X,y  = bc.data, bc.target
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [4]:
# scale the features - 0 mean, 1 variance - recommended when dealing with logistic regression

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [5]:
X_train  = torch.from_numpy(X_train.astype(np.float32))
y_train  = torch.from_numpy(y_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
y_test  = torch.from_numpy(y_test.astype(np.float32))

In [6]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [7]:
# create model

# f = wx+b, sigmoid at the end

class LogisticReg(nn.Module):
    def __init__(self, in_dims):
        super(LogisticReg,self).__init__()
        self.linear = nn.Linear(in_dims,1)
        
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    


In [8]:
model  = LogisticReg(n_features)
loss = nn.BCELoss() # binary cross entropy
optimizer = torch.optim.SGD(model.parameters(),lr = 0.05)


In [ ]:
epochs = 1000
loss_tracker = []
for i in range(epochs):
    # forward pass
    y_pred = model(X_train)
    
    #loss 
    l = loss(y_pred,y_train)
    loss_tracker.append(l)
    # backprop
    l.backward()
    
    # update weights
    optimizer.step()
    
    # zero grad
    optimizer.zero_grad()
    if i%100 ==0:
        print(l.item())
    

In [22]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)
    

tensor(0.9649)
